In [1]:
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import requests
from PIL import Image
from io import BytesIO
import arabic_reshaper
from bidi.algorithm import get_display
import textwrap
from math import ceil
import time

In [2]:
plt.set_cmap('gray')

<Figure size 640x480 with 0 Axes>

# Load Gemma Captions Dataset:

In [3]:
gemma_1 = pd.read_csv("Data\Gemma_captions\Gemma_captions_day1.csv")
gemma_2 = pd.read_csv("Data\Gemma_captions\Gemma_captions_day2.csv")
gemma_3 = pd.read_csv("Data\Gemma_captions\Gemma_captions_day3.csv")
gemma_4 = pd.read_csv("Data\Gemma_captions\Gemma_captions_day4.csv")
gemma_5 = pd.read_csv("Data\Gemma_captions\Gemma_captions_day5.csv")
gemma_6 = pd.read_csv("Data\Gemma_captions\Gemma_caption_day6.csv")
print(f'Gemma_1 has{gemma_1.shape}, Gemma_2 has{gemma_2.shape}, Gemma_3 has{gemma_3.shape},  Gemma_4 has{gemma_4.shape}, Gemma_5 has {gemma_5.shape}, Gemma_6 has {gemma_6.shape}')
gemma_1.head()


Gemma_1 has(250, 2), Gemma_2 has(250, 2), Gemma_3 has(250, 2),  Gemma_4 has(250, 2), Gemma_5 has (250, 2), Gemma_6 has (12, 2)


,image,caption
0,1000268201_693b08cb0e.jpg,بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nتظ...
1,1001773457_577c3a7d70.jpg,في الصورة، يواجه كلبان بعضهما البعض على طريق ت...
2,1002674143_1b742ab4b8.jpg,الصورة تظهر طفلة صغيرة جالسة على العشب أمام قو...
3,1003163366_44323f5815.jpg,بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nرج...
4,1007129816_e794419615.jpg,بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nيظ...


# Concatenate all datasets:

In [4]:
data1 = pd.concat([gemma_1, gemma_2, gemma_3], ignore_index=True)
data2 = pd.concat([gemma_4, gemma_5, gemma_6], ignore_index=True)

df = pd.concat([data1, data2], ignore_index=True)
print(df.shape)
df.head()


(1262, 2)


,image,caption
0,1000268201_693b08cb0e.jpg,بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nتظ...
1,1001773457_577c3a7d70.jpg,في الصورة، يواجه كلبان بعضهما البعض على طريق ت...
2,1002674143_1b742ab4b8.jpg,الصورة تظهر طفلة صغيرة جالسة على العشب أمام قو...
3,1003163366_44323f5815.jpg,بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nرج...
4,1007129816_e794419615.jpg,بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nيظ...


In [5]:
missing_caption = df[df["caption"].isna()]
missing_caption.shape

(34, 2)

In [6]:
df = df.dropna(subset=["caption"]).reset_index(drop=True)
df.shape

(1228, 2)

In [7]:
df.caption[:20]

0     بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nتظ...
1     في الصورة، يواجه كلبان بعضهما البعض على طريق ت...
2     الصورة تظهر طفلة صغيرة جالسة على العشب أمام قو...
3     بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nرج...
4     بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nيظ...
5     الصورة تظهر طفلة صغيرة تتسلق شبكة حمراء في ملع...
6     في الصورة، يركض كلب بوستون تيرير أسود وأبيض عل...
7     بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nكل...
8     الصورة تظهر طفلاً صغيراً يرتدي ملابس زرقاء وبي...
9     بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nكل...
10    في الصورة، يركض كلب رمادي وأبيض في الثلج، ويبد...
11    تُظهر الصورة رسامًا يعرض لوحاته في الغابة المغ...
12    بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nيظ...
13    بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nيظ...
14    بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nكل...
15    في الصورة، يقفز كلب أبيض وأسود في الهواء محاول...
16    الصورة تظهر طفلين يقفان على ضفاف بحيرة، أحدهما...
17    بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n

# Load Gemini Datasets:

In [8]:
gemini_1 = pd.read_csv("Data\Gemini_captions\Gemini_captions_day1.csv")
gemini_2 = pd.read_csv("Data\Gemini_captions\Gemini_captions_day2.csv")
gemini_3 = pd.read_csv("Data\Gemini_captions\Gemini_captions_day3.csv")
gemini_4 = pd.read_csv("Data\Gemini_captions\Gemini_captions_day4.csv")
gemini_5 = pd.read_csv("Data\Gemini_captions\Gemini_captions_day5.csv")
print(f'gemini_1 has{gemini_1.shape}, gemini_2 has{gemini_2.shape}, gemini_3 has{gemini_3.shape}, gemini_4 has{gemini_4.shape}, gemini_5 has{gemini_5.shape}')
gemini_1.head()


gemini_1 has(246, 2), gemini_2 has(245, 2), gemini_3 has(238, 2), gemini_4 has(248, 2), gemini_5 has(247, 2)


,image,caption
0,1001773457_577c3a7d70.jpg,كلبان يلعبان على طريق أسفلتي.
1,1002674143_1b742ab4b8.jpg,تظهر الصورة طفلة صغيرة ملطخة بالألوان تجلس على...
2,1003163366_44323f5815.jpg,يظهر في الصورة رجل مستلقٍ على مقعد وكلبه جالس ...
3,1007129816_e794419615.jpg,رجل يرتدي نظارات وقبعة برتقالية فريدة مصنوعة م...
4,1007320043_627395c3d8.jpg,طفلة صغيرة تتسلق شبكة حبال حمراء في ملعب وتنظر...


# Concatenate all datasets:

In [9]:
data1 = pd.concat([gemini_1, gemini_2], ignore_index=True)
data2 = pd.concat([gemini_3, gemini_4, gemini_5], ignore_index=True)

gemini_df = pd.concat([data1, data2], ignore_index=True)
print(gemini_df.shape)
gemini_df.head()


(1224, 2)


,image,caption
0,1001773457_577c3a7d70.jpg,كلبان يلعبان على طريق أسفلتي.
1,1002674143_1b742ab4b8.jpg,تظهر الصورة طفلة صغيرة ملطخة بالألوان تجلس على...
2,1003163366_44323f5815.jpg,يظهر في الصورة رجل مستلقٍ على مقعد وكلبه جالس ...
3,1007129816_e794419615.jpg,رجل يرتدي نظارات وقبعة برتقالية فريدة مصنوعة م...
4,1007320043_627395c3d8.jpg,طفلة صغيرة تتسلق شبكة حبال حمراء في ملعب وتنظر...


In [10]:
missing_caption = gemini_df[gemini_df["caption"].isna()]
missing_caption.shape

(15, 2)

In [11]:
gemini_df = gemini_df.dropna(subset=["caption"]).reset_index(drop=True)
gemini_df.shape

(1209, 2)

In [12]:
gemini_df.caption[:20]

0                         كلبان يلعبان على طريق أسفلتي.
1     تظهر الصورة طفلة صغيرة ملطخة بالألوان تجلس على...
2     يظهر في الصورة رجل مستلقٍ على مقعد وكلبه جالس ...
3     رجل يرتدي نظارات وقبعة برتقالية فريدة مصنوعة م...
4     طفلة صغيرة تتسلق شبكة حبال حمراء في ملعب وتنظر...
5            كلب بوسطن تيرير يركض بحماس في حديقة خضراء.
6     كلب أبيض يهز جسمه لينفض الماء عن نفسه على الشا...
7     يظهر صبي صغير مبتسم يمشي على الرصيف أمام جدار ...
8           كلب سعيد يقفز فوق جذع شجرة في الهواء الطلق.
9                 كلب رشيق يركض بفرح في الثلج المتناثر.
10    متزلج يمر بجانب رجل يعرض لوحات فنية في غابة ثل...
11    تُظهر الصورة متسلقة صخور في لقطات متتالية أثنا...
12    كلب بني يلعب بحماس على العشب الأخضر بالقرب من ...
13                 كلب أبيض يطارد كرة صفراء في الحديقة.
14    كلب أبيض يقفز عالياً في مرج أخضر لالتقاط كرة ص...
15    يظهر طفل يلوح لبطة على ضفة بحيرة في حديقة حضري...
16    تُظهر الصورة عائلة تجلس بجانب بحيرة مع طفل رضي...
17              كلب أسود يجري في الأمواج ويُحدث 

In [13]:
caption_df = pd.merge(df, gemini_df, on="image", how="inner")
caption_df.columns = ["image", "Gemma_caption", "Gemini_caption"]
caption_df.head()

,image,Gemma_caption,Gemini_caption
0,1001773457_577c3a7d70.jpg,في الصورة، يواجه كلبان بعضهما البعض على طريق ت...,كلبان يلعبان على طريق أسفلتي.
1,1002674143_1b742ab4b8.jpg,الصورة تظهر طفلة صغيرة جالسة على العشب أمام قو...,تظهر الصورة طفلة صغيرة ملطخة بالألوان تجلس على...
2,1003163366_44323f5815.jpg,بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nرج...,يظهر في الصورة رجل مستلقٍ على مقعد وكلبه جالس ...
3,1007129816_e794419615.jpg,بالتأكيد، إليك وصف للصورة في جملة قصيرة:\n\nيظ...,رجل يرتدي نظارات وقبعة برتقالية فريدة مصنوعة م...
4,1007320043_627395c3d8.jpg,الصورة تظهر طفلة صغيرة تتسلق شبكة حمراء في ملع...,طفلة صغيرة تتسلق شبكة حبال حمراء في ملعب وتنظر...


In [14]:
print(f'The data has {caption_df.shape} captions')

The data has (1192, 3) captions


# Captions perprocessing:

In [15]:
import re

def clean_intro(text):
    # Remove leading punctuation/spaces
    text = re.sub(r"^[\s:\-،]*", "", text)
    
    # Remove common intro patterns
    patterns = [
    r"^(?:بالتأكيد(?:،)?(?:\s*إليك(?:\s*وصف(?:\s*للصورة)?)?)?)[:：\s,-]*",  
    r"^(?:في(?:\s+هذه)?\s+الصورة)[:：\s،,-]*",                              
    r"^(?:تُظهر|يظهر|تُظهر|تبدوا|تبدو)[:：\s،,-]*(?:الصورة)?\s*",                 
    r"^(?:الصورة(?:\s+تظهر|\s+تُظهر))[:：\s،,-]*",                          
    r"^(?:في\s+الصوره)[:：\s،,-]*",                                         
    r"^(?:يبدو\s+أن(?:\s+ال)?\s*الصورة)[:：\s،,-]*",                        
    r"^(?:في\s+الصور?ة)[:：\s،,-]*",                                        
    r"(?:في\s+جملة\s+قصيرة[:：\s،,-]*)",      
    r"(?:بجملة\s+قصيرة[:：\s،,-]*)"           
    ]                                   
    
    for p in patterns:
        text = re.sub(p, "", text)
    return text.strip()

In [16]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("[ًٌٍَُِّْـ]", "", text)
    return text

In [17]:
def preprocess_caption(text):
    text = str(text).replace("\n", " ").replace("\\n", " ").strip()
    text = clean_intro(text)
    text = re.sub(r"[A-Za-z0-9]+", "", text)  # remove Latin chars/numbers
    text = normalize_arabic(text)
    text = re.sub(r"\s+", " ", text)  # collapse spaces
    return text.strip()

In [18]:
caption_df["Gemma_caption"] = caption_df["Gemma_caption"].astype(str).apply(preprocess_caption)
caption_df["Gemini_caption"] = caption_df["Gemini_caption"].astype(str).apply(preprocess_caption)

caption_df.head()

,image,Gemma_caption,Gemini_caption
0,1001773457_577c3a7d70.jpg,يواجه كلبان بعضهما البعض علي طريق ترابي، ويبدو...,كلبان يلعبان علي طريق اسفلتي.
1,1002674143_1b742ab4b8.jpg,طفله صغيره جالسه علي العشب امام قوس قزح كبير ر...,تظهر الصوره طفله صغيره ملطخه بالالوان تجلس علي...
2,1003163366_44323f5815.jpg,رجل مستلق علي مقعد في الهواء الطلق، بينما يجلس...,رجل مستلق علي مقعد وكلبه جالس امامه امام سد ماءي.
3,1007129816_e794419615.jpg,يظهر في الصوره رجل يرتدي قبعه غريبه مصنوعه من ...,رجل يرتدي نظارات وقبعه برتقاليه فريده مصنوعه م...
4,1007320043_627395c3d8.jpg,طفله صغيره تتسلق شبكه حمراء في ملعب، وتبدو سعي...,طفله صغيره تتسلق شبكه حبال حمراء في ملعب وتنظر...


# Load Arabic Flickr8k Captions

In [19]:
data = pd.read_csv('Data\Flickr8k_arabic.csv')
data.head()

,image,caption
0,1000268201_693b08cb0e.jpg,طفلة صغيرة تتسلق إلى مسرح خشبي
1,1001773457_577c3a7d70.jpg,كلب أسود وكلب ثلاثي الألوان يلعبان مع بعضهما ا...
2,1002674143_1b742ab4b8.jpg,فتاة صغيرة مغطاة بالطلاء تجلس أمام قوس قزح
3,1003163366_44323f5815.jpg,رجل يجلس على مقعد بينما يجلس كلبه بجانبه
4,1007129816_e794419615.jpg,رجل يرتدي قبعة برتقالية


# Merge real captions with generative caption in one dataset:

In [20]:
caption_data = pd.merge(data, caption_df, on="image", how="inner")
caption_data.columns = ["image","caption", "Gemma_caption", "Gemini_caption"]
caption_data.head()

,image,caption,Gemma_caption,Gemini_caption
0,1001773457_577c3a7d70.jpg,كلب أسود وكلب ثلاثي الألوان يلعبان مع بعضهما ا...,يواجه كلبان بعضهما البعض علي طريق ترابي، ويبدو...,كلبان يلعبان علي طريق اسفلتي.
1,1002674143_1b742ab4b8.jpg,فتاة صغيرة مغطاة بالطلاء تجلس أمام قوس قزح,طفله صغيره جالسه علي العشب امام قوس قزح كبير ر...,تظهر الصوره طفله صغيره ملطخه بالالوان تجلس علي...
2,1003163366_44323f5815.jpg,رجل يجلس على مقعد بينما يجلس كلبه بجانبه,رجل مستلق علي مقعد في الهواء الطلق، بينما يجلس...,رجل مستلق علي مقعد وكلبه جالس امامه امام سد ماءي.
3,1007129816_e794419615.jpg,رجل يرتدي قبعة برتقالية,يظهر في الصوره رجل يرتدي قبعه غريبه مصنوعه من ...,رجل يرتدي نظارات وقبعه برتقاليه فريده مصنوعه م...
4,1007320043_627395c3d8.jpg,طفل يلعب على حبل,طفله صغيره تتسلق شبكه حمراء في ملعب، وتبدو سعي...,طفله صغيره تتسلق شبكه حبال حمراء في ملعب وتنظر...


# Data Preprocessing:

In [21]:
caption_data["Gemma_caption"] = caption_data["Gemma_caption"].astype(str).apply(preprocess_caption)
caption_data["Gemini_caption"] = caption_data["Gemini_caption"].astype(str).apply(preprocess_caption)
caption_data["caption"] = caption_data["caption"].astype(str).apply(preprocess_caption)
caption_data.head()

,image,caption,Gemma_caption,Gemini_caption
0,1001773457_577c3a7d70.jpg,كلب اسود وكلب ثلاثي الالوان يلعبان مع بعضهما ا...,يواجه كلبان بعضهما البعض علي طريق ترابي، ويبدو...,كلبان يلعبان علي طريق اسفلتي.
1,1002674143_1b742ab4b8.jpg,فتاه صغيره مغطاه بالطلاء تجلس امام قوس قزح,طفله صغيره جالسه علي العشب امام قوس قزح كبير ر...,تظهر الصوره طفله صغيره ملطخه بالالوان تجلس علي...
2,1003163366_44323f5815.jpg,رجل يجلس علي مقعد بينما يجلس كلبه بجانبه,رجل مستلق علي مقعد في الهواء الطلق، بينما يجلس...,رجل مستلق علي مقعد وكلبه جالس امامه امام سد ماءي.
3,1007129816_e794419615.jpg,رجل يرتدي قبعه برتقاليه,"رجل يرتدي قبعه غريبه مصنوعه من علب بيره ""-"" مح...",رجل يرتدي نظارات وقبعه برتقاليه فريده مصنوعه م...
4,1007320043_627395c3d8.jpg,طفل يلعب علي حبل,طفله صغيره تتسلق شبكه حمراء في ملعب، وتبدو سعي...,طفله صغيره تتسلق شبكه حبال حمراء في ملعب وتنظر...


In [22]:
caption_data.shape

(1192, 4)

# Save the caption data:

In [ ]:
caption_data.to_csv('Data\Captions.csv', index=False)
